# Analysis for *Generating Event Descriptions*

In [ ]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import arviz

from typing import Literal, Callable
from dataclasses import dataclass
from glob import glob
from scipy.stats import spearmanr
from patsy import dmatrix
from cmdstanpy import CmdStanModel, CmdStanMCMC, from_csv

## Experiment 1: Naturalness

We begin analysis of the naturalness experiment by loading and combining the naturalness datasets into a single `pandas.DataFrame`.

In [ ]:
nat_fnames = [
    "naturalness_llama-propbanksenses.csv",
    "naturalness_llama-llamasenses.csv",
    "naturalness_reddit.csv",
    "naturalness_manual.csv"
]

data_nat = []

for fname in nat_fnames:
    df = pd.read_csv(f"data/{fname}")

    generation_method = fname.replace("naturalness_", "").strip(".csv")

    if generation_method == "manual":
        df["list_type"] = "manual"
        df["generation_method_type"] = "manual"
        df["generation_method_subtype"] = "manual"
        df["generation_method_subsubtype"] = "manual"
        df = df.query("sentence_type == 'target'")
        df["generation_method"] = "manual_" + df.naturalness + "_" + df.typicality
    else:
        df["list_type"] = generation_method
        df["generation_method_type"] = df.sentence_type.map(lambda x: "manual" if x == "calibration" else "automated")
        df["generation_method_subtype"] = df.sentence_type.map(
            lambda x: "manual" if x == "calibration" else generation_method.split("-")[0]
        )
        df["generation_method_subsubtype"] = df.sentence_type.map(
            lambda x: "manual" if x == "calibration" else generation_method
        )
        df["generation_method"] = df[["sentence_type", "naturalness", "typicality"]].agg(
            lambda x: "manual_" + x.naturalness + "_" + x.typicality if x.sentence_type == "calibration" else generation_method,
            axis=1
        )

    data_nat.append(df)

data_nat = pd.concat(data_nat)

data_nat["surprisal_z"] = (data_nat.surprisal - data_nat.surprisal.mean())/data_nat.surprisal.std()
data_nat["freq_z"] = (data_nat.freq - data_nat.freq.mean())/data_nat.freq.std()

data_nat

### Participant filtration

Some participants did not complete the survey. We remove these participants.

In [ ]:
def filter_participants(data, expected_counts: dict[str, int]) -> pd.DataFrame:
    """Filter participants who did not complete the task"""
    actual_counts = data.groupby("list_type").rater_id.value_counts().reset_index()

    exclude_participants = [
        r.rater_id for _, r in actual_counts.iterrows()
        if r["count"] != expected_counts[r.list_type]
    ]

    return data[~data.rater_id.isin(exclude_participants)]

In [ ]:
expected_counts_nat = {
    "llama-llamasense": 97,
    "llama-propbanksense": 97,
    "manual": 124,
    "reddit": 72
}

data_nat = filter_participants(data_nat, expected_counts_nat)

data_nat

### Model fitting

To analyze the data, we use a generlized linear mixed effects model with an ordered beta link. This model is implemented in `scripts/analysis/models/ordered-beta.stan`. 

In [ ]:
model = CmdStanModel(stan_file="scripts/analysis/models/ordered-beta.stan")

To fit this model to our data, we need to map it into the format assumd by the model's `data` block.

```stan
data {
  int<lower=1> N_resp;                             // number of responses
  int<lower=1> N_subj;                             // number of subjects
  int<lower=1> N_verb;                             // number of verbs
  int<lower=1> N_sense;                            // number of senses
  int<lower=1> N_item;                             // number of items
  int<lower=1> N_fixed;                            // number of fixed predictors
  int<lower=1> N_by_subj;                          // number of random by-subject predictors
  int<lower=1> N_by_verb;                          // number of random by-verb predictors
  int<lower=1> N_by_sense;                         // number of random by-sense predictors
  int<lower=1> N_by_item;                          // number of random by-item predictors
  matrix[N_resp,N_fixed] fixed_predictors;         // predictors including intercept
  matrix[N_resp,N_by_subj] by_subj_predictors;     // by-subject predictors including intercept
  matrix[N_resp,N_by_verb] by_verb_predictors;     // by-verb predictors including intercept
  matrix[N_resp,N_by_sense] by_sense_predictors;   // by-sense predictors including intercept
  matrix[N_resp,N_by_item] by_item_predictors;     // by-item predictors including intercept
  array[N_resp] int<lower=1,upper=N_subj> subj;    // subject who gave response n
  array[N_resp] int<lower=1,upper=N_verb> verb;    // verb corresponding to response n
  array[N_resp] int<lower=1,upper=N_sense> sense;  // sense corresponding to response n
  array[N_resp] int<lower=1,upper=N_item> item;    // item corresponding to response n
  array[N_resp] int<lower=1,upper=3> resp_bin;     // whether a response is 0=1, (0, 1)=1, or 1=2
  array[N_resp] real<lower=0,upper=1> resp;        // [0, 1] responses                                    
}
```

In [ ]:
def bin_response(x: float) -> int:
    """Bin the response by whether it is an endpoint (0, 1) or not."""
    if x == 0.0:
        return 1
    elif x == 1.0:
        return 3
    else:
        return 2

def prepare_data(
    data: pd.DataFrame, 
    fixed_formula: str,
    by_subj_formula: str,
    by_verb_formula: str,
    by_sense_formula: str,
    by_item_formula: str,
    item_cols: list[str],
    sense_cols: list[str],
    subj_cols: list[str] = ["rater_id"], 
    verb_cols: list[str] = ["verb"], 
    resp_col: str = "rating",
) -> tuple[dict[str, int | np.ndarray], np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    # scale the responses to [0, 1]
    resp = data[resp_col] / 100

    # bin the responses
    resp_bin = resp.map(bin_response)

    # construct the design matrices
    fixed_predictors = dmatrix(fixed_formula, data, return_type="dataframe")
    by_subj_predictors = dmatrix(by_subj_formula, data, return_type="dataframe")
    by_verb_predictors = dmatrix(by_verb_formula, data, return_type="dataframe")
    by_sense_predictors = dmatrix(by_sense_formula, data, return_type="dataframe")
    by_item_predictors = dmatrix(by_item_formula, data, return_type="dataframe")

    # hash the items and subjects
    subjid = data[subj_cols].agg('-'.join, axis=1).astype("category")
    verbid = data[verb_cols].agg('-'.join, axis=1).astype("category")
    senseid = data[sense_cols].agg('-'.join, axis=1).astype("category")
    itemid = data[item_cols].agg('-'.join, axis=1).astype("category")
    
    # determine whether the subject is a discrete responder
    disc_responder = data[[resp_col]].isin([0, 100])
    disc_responder["subjid"] = subjid
    disc_responder = disc_responder.groupby("subjid")[resp_col].all()

    data_stan = {
        "N_resp": data.shape[0],
        "N_subj": subjid.cat.codes.max() + 1,
        "N_verb": verbid.cat.codes.max() + 1,
        "N_sense": senseid.cat.codes.max() + 1,
        "N_item": itemid.cat.codes.max() + 1,
        "N_fixed": fixed_predictors.shape[1],
        "N_by_subj": by_subj_predictors.shape[1],
        "N_by_verb": by_verb_predictors.shape[1],
        "N_by_sense": by_sense_predictors.shape[1],
        "N_by_item": by_item_predictors.shape[1],
        "fixed_predictors": fixed_predictors.values,
        "by_subj_predictors": by_subj_predictors.values,
        "by_verb_predictors": by_verb_predictors.values,
        "by_sense_predictors": by_sense_predictors.values,
        "by_item_predictors": by_item_predictors.values,
        "subj": subjid.cat.codes.values + 1,
        "verb": verbid.cat.codes.values + 1,
        "sense": senseid.cat.codes.values + 1,
        "item": itemid.cat.codes.values + 1,
        "resp_bin": resp_bin.values.astype(int),
        "resp": resp.values
    }

    return (
        data_stan, 
        fixed_predictors.columns.values, 
        by_subj_predictors.columns.values,
        by_verb_predictors.columns.values,
        by_sense_predictors.columns.values,
        by_item_predictors.columns.values,
    )

The estimates for the distribution over fixed and random effects reported in the paper are extracted from the model fits using:

In [ ]:
def fixed_coef_stats(fit: CmdStanMCMC, coef_names: list[str]) -> pd.DataFrame:
    fixed_coefs = fit.stan_variable("fixed_coefs")

    posterior_means = fixed_coefs.mean(axis=0)
    
    stats = pd.DataFrame(
        np.quantile(fixed_coefs, [0.025, 0.05, 0.95,  0.975], axis=0).T, 
        index=coef_names,
        columns=["2.5%", "5%", "95%", "97.5%"]
    )

    stats["post_mean"] = posterior_means
    
    stats["p"] = np.mean(
        np.sign(posterior_means)[None,:] != np.sign(fixed_coefs), 
        axis=0
    )

    return stats[["post_mean", "2.5%", "5%", "95%", "97.5%", "p"]]

def random_coef_stats(
    fit: CmdStanMCMC, 
    var_name: Literal[
        "subj_cov", "subj_corr", 
        "verb_cov", "verb_corr", 
        "sense_cov", "sense_corr", 
        "item_cov", "item_corr"
    ], 
    coef_names: list[str]
) -> pd.DataFrame:
    matrices = fit.stan_variable(var_name)
    posterior_means = pd.DataFrame(
        matrices.mean(axis=0),
        index=coef_names,
        columns=coef_names
    )

    return posterior_means


And the actual model fits are done using:

In [ ]:
@dataclass
class ModelResults:
    fit: CmdStanMCMC
    fixed_coefs: pd.DataFrame
    subj_cov:  pd.DataFrame
    verb_cov:  pd.DataFrame
    sense_cov: pd.DataFrame
    item_cov:  pd.DataFrame
    subj_corr: pd.DataFrame
    verb_corr: pd.DataFrame
    sense_corr: pd.DataFrame
    item_corr: pd.DataFrame

def fit_hmc(
    data: pd.DataFrame,
    fixed_formula: str, 
    by_subj_formula: str,
    by_verb_formula: str,
    by_sense_formula: str,
    by_item_formula: str,  
    item_cols: list[str],
    subj_cols: list[str],
    verb_cols: list[str],
    sense_cols: list[str], 
    seed: int = 30298,
    **kwargs
) -> ModelResults:
    data_stan, fixed_predictors, by_subj_predictors, by_verb_predictors, by_sense_predictors, by_item_predictors = prepare_data(
        data,
        fixed_formula=fixed_formula,
        by_subj_formula=by_subj_formula,
        by_verb_formula=by_verb_formula,
        by_sense_formula=by_sense_formula,
        by_item_formula=by_item_formula, 
        subj_cols=subj_cols,
        verb_cols=verb_cols,
        sense_cols=sense_cols,
        item_cols=item_cols
    )

    fit = model.sample(
        data=data_stan,
        seed=seed,
        **kwargs
    )

    return ModelResults(
        fit = fit, 
        fixed_coefs = fixed_coef_stats(fit, fixed_predictors),
        subj_cov = random_coef_stats(fit, "subj_cov", by_subj_predictors),
        verb_cov = random_coef_stats(fit, "verb_cov", by_verb_predictors),
        sense_cov = random_coef_stats(fit, "sense_cov", by_sense_predictors),
        item_cov = random_coef_stats(fit, "item_cov", by_item_predictors),
        subj_corr = random_coef_stats(fit, "subj_corr", by_subj_predictors),
        verb_corr = random_coef_stats(fit, "verb_corr", by_verb_predictors),
        sense_corr = random_coef_stats(fit, "sense_corr", by_sense_predictors),
        item_corr = random_coef_stats(fit, "item_corr", by_item_predictors)
    )

Because we don't want ot have to rerun the models if we already have them cached, we'll also define a method for loading a fit from CSVs dumped by `cmdstanpy`.

In [ ]:
def load_fit(
    path: str, 
    data: pd.DataFrame,
    fixed_formula: str, 
    by_subj_formula: str,
    by_verb_formula: str,
    by_sense_formula: str,
    by_item_formula: str,  
    item_cols: list[str],
    subj_cols: list[str],
    verb_cols: list[str],
    sense_cols: list[str],
) -> ModelResults:
    _, fixed_predictors, by_subj_predictors, by_verb_predictors, by_sense_predictors, by_item_predictors = prepare_data(
        data,
        fixed_formula=fixed_formula,
        by_subj_formula=by_subj_formula,
        by_verb_formula=by_verb_formula,
        by_sense_formula=by_sense_formula,
        by_item_formula=by_item_formula, 
        subj_cols=subj_cols,
        verb_cols=verb_cols,
        sense_cols=sense_cols,
        item_cols=item_cols
    )

    fit = from_csv(path, method="sample")

    return ModelResults(
        fit = fit, 
        fixed_coefs = fixed_coef_stats(fit, fixed_predictors),
        subj_cov = random_coef_stats(fit, "subj_cov", by_subj_predictors),
        verb_cov = random_coef_stats(fit, "verb_cov", by_verb_predictors),
        sense_cov = random_coef_stats(fit, "sense_cov", by_sense_predictors),
        item_cov = random_coef_stats(fit, "item_cov", by_item_predictors),
        subj_corr = random_coef_stats(fit, "subj_corr", by_subj_predictors),
        verb_corr = random_coef_stats(fit, "verb_corr", by_verb_predictors),
        sense_corr = random_coef_stats(fit, "sense_corr", by_sense_predictors),
        item_corr = random_coef_stats(fit, "item_corr", by_item_predictors)
    )

We use the following sampler parameters throughout.

In [ ]:
sampler_params = {
    "iter_warmup": 2500, 
    "iter_sampling": 2500
}

And to enforce a particular coding of the generation levels, we specify that ordering as a list that will be passed to `patsy`.

In [ ]:
generation_levels = [
    'manual_natural_typical', 'manual_natural_atypical', 
    'manual_unnatural_typical', 'manual_unnatural_atypical',
    'reddit', 'llama-propbanksense', 'llama-llamasense'
]

Finally, we actually fit the model (or load it if we have a cached fit).

In [ ]:
if glob("fits/nat/base/*.csv"):
    results_nat = load_fit(
        "fits/nat/base/*.csv", 
        data_nat,
        fixed_formula="~ 1 + C(generation_method, levels=generation_levels)",
        by_subj_formula="~ 1", # cannot fit anything bigger, because not all subjects saw items from every generation method
        by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
        by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
        by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"],
    )
else:
    results_nat = fit_hmc(
        data_nat,
        fixed_formula="~ 1 + C(generation_method, levels=generation_levels)",
        by_subj_formula="~ 1", # cannot fit anything bigger, because not all subjects saw items from every generation method
        by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
        by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
        by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"],
        output_dir="fits/nat/base",
        **sampler_params
    )

    results_nat.fit.diagnose()

    # handles a bug in cmdstanpy.from_csv that I suspect has to do with a version change in STAN
    # basically, I *think* STAN used to dump the save_warmup flag as an int, but now it uses a boolean
    !find . -type f -wholename './fits/nat/base/*.csv' | xargs sed -i 's/save_warmup = false/save_warmup = 0/g'

We can see the fixed effects coefficient estimates by looking at the `fixed_coefs` attribute of the `ModelResults`.

In [ ]:
results_nat.fixed_coefs

We generate the $\LaTeX$ table used in the paper using the following code.

In [ ]:
def print_fixed_coef_tabular(fit: ModelResults, coef_map: Callable[str, str]) -> None:
    print(r"\begin{tabular}{rcccr}")

    print(r"\toprule")
    print(r"      & \textbf{Post. mean} & \textbf{2.5\%} & \textbf{97.5\%} & \textbf{Post.} $p$ \\")
    print(r"\midrule")

    for coef, row in fit.fixed_coefs.iterrows():
        if row.p < 0.001:
            print(f"{coef_map(coef)} & {row.post_mean:2.2f} & {row['2.5%']:2.2f} & {row['97.5%']:2.2f} & $<$ 0.01 \\\\")
        else:    
            print(f"{coef_map(coef)} & {row.post_mean:2.2f} & {row['2.5%']:2.2f} & {row['97.5%']:2.2f} & {row.p:2.2f} \\\\")

    print(r"\bottomrule")
    print(r"\end{tabular}")

coef_map = {
    "manual_unnatural_typical": "Manual (Unnatural \\& Typical)",
    "manual_natural_typical": "Manual (Natural \\& Typical )",
    "manual_unnatural_atypical": "Manual (Unnatural \\& Atypical)",
    "manual_natural_atypical": "Manual (Natural \\& Atypical)",
    "reddit": "Corpus",
    "llama-propbanksense": "LM with PropBank senses",
    "llama-llamasense": "LM with LM senses",
    "llama": "LM"
}

def process_coef_name_nat_typ(coef: str) -> str:
    if coef == "Intercept":
        return coef
    else:
        return " $\\times$ ".join(
            coef_map[v] for v in re.findall(
                "C\(generation_method, levels=generation_levels\)\[T\.(.*)\]", 
                coef
            )
        )

In [ ]:
print_fixed_coef_tabular(results_nat, process_coef_name_nat_typ)

### Cutpoint distributions

In the caption of Table 8 of the paper, we report estimates of the posterior distributions over cutpoints. These estimates are calculated below.

In [ ]:
def cutpoint_stats(fit: CmdStanMCMC) -> tuple[np.ndarray, np.ndarray]: 
    cutpoint0 = fit.stan_variable("cutpoint0")
    cutpoint1 = cutpoint0 + np.exp(fit.stan_variable("interval_size_logmean"))

    cutpoint0_stats = np.round(np.quantile(cutpoint0, [0.025, 0.5, 0.975]), 2)
    cutpoint1_stats = np.round(np.quantile(cutpoint1, [0.025, 0.5, 0.975]), 2)

    print("Cutpoint 0:", cutpoint0_stats[1], f"(95\\% CI = [{cutpoint0_stats[0]}, {cutpoint0_stats[2]}])")
    print("Cutpoint 1:", cutpoint1_stats[1], f"(95\\% CI = [{cutpoint1_stats[0]}, {cutpoint1_stats[2]}])")

    return cutpoint0_stats, cutpoint1_stats

In [ ]:

_ = cutpoint_stats(results_nat.fit)

### Difference between manual generation and automatic generation

In Section 4.5 of the paper, we discuss the differences between the ratings for manually generated items that were constructed to be natural and those for automatically generated items. The difference between the manually generated natural, typical items can be read directly off the coefficient estimates. The difference between the manually generated natural, typical items and the automatically generated items needs to be calculated from the posterior samples.

To assess whether the automatic generation methods produce examples that are more natural than the manually generated natural, atypical examples, we compute the posterior distribution of the difference in the automatic effects and the manual natural-atypical effect, then test whether it is greater than 0. The proportion of samples on which it is greate than 0 gives us the posterior $p$ that the automatic effects are indeed more positive.

In [ ]:
natural_atypical_effect = results_nat.fit.stan_variable("fixed_coefs")[:,[1]]
automatic_effects = results_nat.fit.stan_variable("fixed_coefs")[:,4:]

mean_difference_gt0 = ((automatic_effects - natural_atypical_effect) > 0).mean(0)

for coef_name, d in zip(generation_levels[4:], mean_difference_gt0):
    if d > 0.99:
        print(coef_map[coef_name], "(posterior $p >$ 0.99)")
    else:
        print(coef_map[coef_name], f"(posterior $p = $ {np.round(d, 2)})")

### Predicting naturalness from frequency

In Section 4.6 of the paper, we report regressions assessing whether our automated methods are frequency-sensitiv with respect to naturalness, finding that they are not. These regressions are conducted below. 

In [ ]:
data_nat_freqsub = data_nat.query('generation_method_type != "manual" or generation_method == "manual_natural_typical"')

data_nat_freqsub

In [ ]:
generation_subtype_levels = [
    "manual", "reddit", "llama"
]

In [ ]:
if glob("fits/nat/frequency/*.csv"):
    results_nat_freq = load_fit(
        "fits/nat/frequency/*.csv",
        data_nat_freqsub,
        fixed_formula="~ 1 + C(generation_method_subtype, levels=generation_subtype_levels) * freq_z",
        by_subj_formula="~ 1 + freq_z", # cannot fit anything bigger, because not all subjects saw items from every generation method
        by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
        by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
        by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"]
    )
else:
    results_nat_freq = fit_hmc(
        data_nat_freqsub,
        fixed_formula="~ 1 + C(generation_method_subtype, levels=generation_subtype_levels) * freq_z",
        by_subj_formula="~ 1 + freq_z", # cannot fit anything bigger, because not all subjects saw items from every generation method
        by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
        by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
        by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"],
        output_dir="fits/nat/frequency",
        **sampler_params
    )

    results_nat_freq.fit.diagnose()

    # handles a bug in cmdstanpy.from_csv that I suspect has to do with a version change in STAN
    # basically, I *think* STAN used to dump the save_warmup flag as an int, but now it uses a boolean
    !find . -type f -wholename './fits/nat/frequency/*.csv' | xargs sed -i 's/save_warmup = false/save_warmup = 0/g'

In [ ]:
results_nat_freq.fixed_coefs

In [ ]:
coef_map["freq_z"] = "Frequency"

def process_coef_name_freq(coef: str) -> str:
    if coef == "Intercept":
        return coef
    else:
        return " $\\times$ ".join(
            coef_map[v] for vs in re.findall(
                "(?:C\(generation_method_subtype, levels=generation_subtype_levels\)\[T\.(.*?)\])?:?(.*)", 
                coef
            ) for v in vs if v
        )

print_fixed_coef_tabular(results_nat_freq, process_coef_name_freq)

In [ ]:
_ = cutpoint_stats(results_nat_freq.fit)

## Experiment 2: Typicality

We begin analysis of the typicality experiment by loading and combining the typicality datasets into a single `pandas.DataFrame`.

In [ ]:
typ_fnames = [
    "typicality_llama-propbanksenses.csv",
    "typicality_llama-llamasenses.csv",
    "typicality_reddit.csv",
    "typicality_manual.csv"
]

data_typ = []

for fname in typ_fnames:
    df = pd.read_csv(f"data/{fname}")

    generation_method = fname.replace("typicality_", "").strip(".csv")

    if generation_method == "manual":
        df["list_type"] = "manual"
        df["generation_method_type"] = "manual"
        df["generation_method_subtype"] = "manual"
        df["generation_method_subsubtype"] = "manual"
        df = df.query("sentence_type == 'target'")
        df["generation_method"] = "manual_" + df.naturalness + "_" + df.typicality
    else:
        df["list_type"] = generation_method
        df["generation_method_type"] = df.sentence_type.map(lambda x: "manual" if x == "calibration" else "automated")
        df["generation_method_subtype"] = df.sentence_type.map(
            lambda x: "manual" if x == "calibration" else generation_method.split("-")[0]
        )
        df["generation_method_subsubtype"] = df.sentence_type.map(
            lambda x: "manual" if x == "calibration" else generation_method
        )
        df["generation_method"] = df[["sentence_type", "naturalness", "typicality"]].agg(
            lambda x: "manual_" + x.naturalness + "_" + x.typicality if x.sentence_type == "calibration" else generation_method,
            axis=1
        )

    data_typ.append(df)

data_typ = pd.concat(data_typ)

data_typ["rating_z"] = data_typ.groupby("rater_id").rating.transform(
    lambda x: (x - x.mean())/x.std()
)

data_typ["surprisal_z"] = (data_typ.surprisal - data_typ.surprisal.mean())/data_typ.surprisal.std()
data_typ["freq_z"] = (data_typ.freq - data_typ.freq.mean())/data_typ.freq.std()

data_typ

### Participant filtration

Some participants did not complete the survey. We remove these participants.

In [ ]:
expected_counts_typ = {
    "llama-llamasense": 97,
    "llama-propbanksense": 97,
    "manual": 124,
    "reddit": 72
}

data_typ = filter_participants(data_typ, expected_counts_typ)

data_typ

### Model fitting

We conduct the typicality model fits using the same setup as we used for naturalness.

In [ ]:
if glob("fits/typ/base/*.csv"):
    results_typ = load_fit(
        "fits/typ/base/*.csv", 
        data_nat,
        fixed_formula="~ 1 + C(generation_method, levels=generation_levels)",
        by_subj_formula="~ 1", # cannot fit anything bigger, because not all subjects saw items from every generation method
        by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
        by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
        by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"],
    )
else:
    results_typ = fit_hmc(
        data_typ,
        fixed_formula="~ 1 + C(generation_method, levels=generation_levels)",
        by_subj_formula="~ 1", # cannot fit anything bigger, because not all subjects saw items from every generation method
        by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
        by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
        by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"],
        output_dir="fits/typ/base/",
        **sampler_params
    )

    results_typ.fit.diagnose()

    # handles a bug in cmdstanpy.from_csv that I suspect has to do with a version change in STAN
    # basically, I *think* STAN used to dump the save_warmup flag as an int, but now it uses a boolean
    !find . -type f -wholename './fits/typ/base/*.csv' | xargs sed -i 's/save_warmup = false/save_warmup = 0/g'

In [ ]:
results_typ.fixed_coefs

In [ ]:
print_fixed_coef_tabular(results_typ, process_coef_name_nat_typ)

In [ ]:
_ = cutpoint_stats(results_typ.fit)

### Predicting typicality from frequency

In Section 5.6 of the paper, we report regressions assessing whether our automated methods are frequency-sensitive with respect to typicality, finding that they are not. These regressions are conducted below. 

In [ ]:
data_typ_freqsub = data_typ.query('generation_method_type != "manual" or generation_method == "manual_natural_typical"')

data_typ_freqsub

In [ ]:
if glob("fits/typ/frequency/*.csv"):
    results_typ_freq = load_fit(
        "fits/typ/frequency/*.csv",
        data_typ_freqsub,
        fixed_formula="~ 1 + C(generation_method_subtype, levels=generation_subtype_levels) * freq_z",
        by_subj_formula="~ 1 + freq_z", # cannot fit anything bigger, because not all subjects saw items from every generation method
        by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
        by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
        by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"]
    )
else:
    results_typ_freq = fit_hmc(
        data_typ_freqsub,
        fixed_formula="~ 1 + C(generation_method_subtype, levels=generation_subtype_levels) * freq_z",
        by_subj_formula="~ 1 + freq_z", # cannot fit anything bigger, because not all subjects saw items from every generation method
        by_verb_formula="~ 1", # cannot fit anything bigger, because not all verbs show up with each generation method
        by_sense_formula="~ 1", # cannot fit anything bigger, because not all verb senses show up with each generation method
        by_item_formula="~ 1", # cannot fit anything bigger, because items are specific to generation method
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"],
        output_dir="fits/typ/frequency",
        **sampler_params
    )

    results_typ_freq.fit.diagnose()

    # handles a bug in cmdstanpy.from_csv that I suspect has to do with a version change in STAN
    # basically, I *think* STAN used to dump the save_warmup flag as an int, but now it uses a boolean
    !find . -type f -wholename './fits/typ/frequency/*.csv' | xargs sed -i 's/save_warmup = false/save_warmup = 0/g'

In [ ]:
results_typ_freq.fixed_coefs

In [ ]:
print_fixed_coef_tabular(results_typ_freq, process_coef_name_freq)

In [ ]:
_ = cutpoint_stats(results_typ_freq.fit)

### Predict naturalness from typicality

In Section 5.6 of the paper, we investigate the hypothesis that typicality plays a role in the degraded naturalness of the automatically generated sentences, finding that it does. These regressions are conducted below.

In [ ]:
data_typ_mean = data_typ.pivot_table(
    index="sentence", values="rating_z", aggfunc=np.mean
).rename(
    columns={"rating_z": "typicality_rating"}
).reset_index()

data_typ_mean

In [ ]:
data_nat_typ = pd.merge(data_nat, data_typ_mean, how="left")

data_nat_typ

In [ ]:
if glob("fits/nat/typicality/*.csv"):
    results_nat_typ = load_fit(
        "fits/nat/typicality/*.csv", 
        data_nat_typ,
        fixed_formula="~ 1 + typicality_rating",
        by_subj_formula="~ 1 + typicality_rating",
        by_verb_formula="~ 1 + typicality_rating",
        by_sense_formula="~ 1 + typicality_rating", 
        by_item_formula="~ 1", # cannot fit anything bigger, because each sentence has only one typicality rating
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"],
    )
else:
    results_nat_typ = fit_hmc(
        data_nat_typ,
        fixed_formula="~ 1 + typicality_rating",
        by_subj_formula="~ 1 + typicality_rating",
        by_verb_formula="~ 1 + typicality_rating",
        by_sense_formula="~ 1 + typicality_rating", 
        by_item_formula="~ 1", # cannot fit anything bigger, because each sentence has only one typicality rating
        item_cols = ["sentence"],
        subj_cols = ["rater_id"],
        verb_cols = ["verb"],
        sense_cols = ["verb", "sense"],
        output_dir="fits/nat/typicality",
        **sampler_params
    )

    results_nat_typ.fit.diagnose()

    # handles a bug in cmdstanpy.from_csv that I suspect has to do with a version change in STAN
    # basically, I *think* STAN used to dump the save_warmup flag as an int, but now it uses a boolean
    !find . -type f -wholename './fits/nat/typicality/*.csv' | xargs sed -i 's/save_warmup = false/save_warmup = 0/g'

In [ ]:
results_nat_typ.fixed_coefs

In [ ]:
post_mean = np.round(results_nat_typ.fixed_coefs.post_mean[1], 2)
cilo = np.round(results_nat_typ.fixed_coefs["2.5%"][1], 2)
cihi = np.round(results_nat_typ.fixed_coefs["97.5%"][1], 2)

print(f"($\\beta=${post_mean}, 95\\% CrI=[{cilo}, {cihi}])")

## Experiment 3: Distinctiveness

In [ ]:
diff_fnames = [
    "difference_llama-propbanksenses.csv",
    "difference_llama-llamasenses.csv",
    "difference_reddit.csv",
    "difference_manual.csv"
]

data_diff = []

for fname in diff_fnames:
    df = pd.read_csv(f"data/{fname}")

    generation_method = fname.replace("difference_", "").strip(".csv")

    if generation_method == "manual":
        df["list_type"] = "manual"
        df["generation_method_type"] = "manual"
        df["generation_method_subtype"] = "manual"
        df["generation_method_subsubtype"] = "manual"
        df = df.query("pair_type == 'target'")
        df["generation_method"] = "manual"
    else:
        df["list_type"] = generation_method
        df["generation_method_type"] = df.pair_type.map(lambda x: "manual" if x == "calibration" else "automated")
        df["generation_method_subtype"] = df.pair_type.map(
            lambda x: "manual" if x == "calibration" else generation_method.split("-")[0]
        )
        df["generation_method_subsubtype"] = df.pair_type.map(
            lambda x: "manual" if x == "calibration" else generation_method
        )
        df["generation_method"] = df["generation_method_subsubtype"]

    data_diff.append(df)

data_diff = pd.concat(data_diff)

data_diff["surprisal1_z"] = (data_diff.surprisal1 - data_diff.surprisal1.mean())/data_diff.surprisal1.std()
data_diff["surprisal2_z"] = (data_diff.surprisal2 - data_diff.surprisal2.mean())/data_diff.surprisal2.std()

data_diff["freq_z"] = (data_diff.freq - data_diff.freq.mean())/data_diff.freq.std()

data_diff["logfreq"] = np.log1p(data_diff["freq"].fillna(0))
data_diff["logfreq_z"] = (data_diff.logfreq - data_diff.logfreq.mean())/data_diff.logfreq.std()

if data_diff.sense1.isnull().any() or data_diff.sense2.isnull().any():
    print("Inserting random dummy senses.")
    data_diff["sense1"] = np.random.choice(100, size=data_diff.shape[0]).astype(str)
    data_diff["sense2"] = np.random.choice(100, size=data_diff.shape[0]).astype(str)

if (data_diff.comparison == "filler").any():
    data_diff_automated_calibration = data_diff[data_diff.pair_type=="filler"].drop(columns=["comparison"])

    data_diff_automated_calibration = pd.merge(
        data_diff_automated_calibration ,
        data_diff[data_diff.generation_method=="manual"][["sentence1", "sentence2", "comparison"]].drop_duplicates()
    )

    data_diff = pd.concat(
        [data_diff[data_diff.pair_type != "filler"], data_diff_automated_calibration],
        axis=0
    )

data_diff

In [ ]:
expected_counts_diff = {
    "llama-llamasense": 68,
    "llama-propbanksense": 68,
    "manual": 62,
    "reddit": 68
}

data_diff = filter_participants(data_diff, expected_counts_diff)

data_diff

In [ ]:
generation_levels = [
    'manual', 'reddit', 
    'llama-propbanksense', 'llama-llamasense'
]

comparison_levels = ['same', 'different']  

In [ ]:
if glob("fits/pairdiff/base/*.csv"):
    results_diff = load_fit(
        "fits/pairdiff/base/*.csv", 
        data_diff,
        fixed_formula="~ 1 + C(comparison, levels=comparison_levels) * C(generation_method, levels=generation_levels)",
        by_subj_formula="~ 1 + C(comparison, levels=comparison_levels)",
        by_verb_formula="~ 1 + C(comparison, levels=comparison_levels)",
        by_sense_formula="~ 1", 
        by_item_formula="~ 1",
        item_cols=["sentence1", "sentence2"],
        sense_cols = ["verb", "sense1", "sense2"],
        subj_cols=["rater_id"],
        verb_cols = ["verb"],
    )
else:
    results_diff = fit_hmc(
        data_diff,
        fixed_formula="~ 1 + C(comparison, levels=comparison_levels) * C(generation_method, levels=generation_levels)",
        by_subj_formula="~ 1 + C(comparison, levels=comparison_levels)",
        by_verb_formula="~ 1 + C(comparison, levels=comparison_levels)",
        by_sense_formula="~ 1", 
        by_item_formula="~ 1",
        item_cols=["sentence1", "sentence2"],
        sense_cols = ["verb", "sense1", "sense2"],
        subj_cols=["rater_id"],
        verb_cols = ["verb"],
        output_dir="fits/pairdiff/base/",
        **sampler_params
    )

    results_diff.fit.diagnose()

    # handles a bug in cmdstanpy.from_csv that I suspect has to do with a version change in STAN
    # basically, I *think* STAN used to dump the save_warmup flag as an int, but now it uses a boolean
    !find . -type f -wholename './fits/pairdiff/base/*.csv' | xargs sed -i 's/save_warmup = false/save_warmup = 0/g'



In [ ]:
results_diff.fixed_coefs

In [ ]:
def process_coef_name_pairdiff(coef: str) -> str:
    coef_map = {
        "manual": "Manual",
        "reddit": "Corpus",
        "llama-propbanksense": "LM with PropBank senses",
        "llama-llamasense": "LM with LM senses",
        "different": "Different Sense",
        "same": "Same Sense"
    }

    if coef == "Intercept":
        return coef
    else:
        return " $\\times$ ".join(
            coef_map[v] for v in re.findall(
                "C\(.*?\)\[T\.(.*?)\]", 
                coef
            )
        )

print_fixed_coef_tabular(results_diff, process_coef_name_pairdiff)

In [ ]:
intercept = results_diff.fit.stan_variable("fixed_coefs")[:,0]
different_effect = results_diff.fit.stan_variable("fixed_coefs")[:,1]
corpus_effect = results_diff.fit.stan_variable("fixed_coefs")[:,2]
corpus_different_effect = results_diff.fit.stan_variable("fixed_coefs")[:,5]

manual_different = intercept + different_effect
corpus_different = intercept + different_effect + corpus_effect + corpus_different_effect

((corpus_different - manual_different) < 0).mean()

In [ ]:
pbsense_effect = results_diff.fit.stan_variable("fixed_coefs")[:,3]
lmsense_effect = results_diff.fit.stan_variable("fixed_coefs")[:,4]
pbsense_different_effect = results_diff.fit.stan_variable("fixed_coefs")[:,6]
lmsense_different_effect = results_diff.fit.stan_variable("fixed_coefs")[:,7]

pbsense_different = intercept + different_effect + pbsense_effect + pbsense_different_effect
lmsense_different = intercept + different_effect + lmsense_effect + lmsense_different_effect

((lmsense_different - pbsense_different) < 0).mean()

In [ ]:
((pbsense_different - manual_different) < 0).mean()

In [ ]:
((corpus_different - pbsense_different) < 0).mean()

In [ ]:
_ = cutpoint_stats(results_diff.fit)

In [ ]:
cutpoint1_pairdiff = cutpoint0_pairdiff + np.exp(fit_pairdiff["fit"].stan_variable("interval_size_logmean"))

print(
    np.round(cutpoint1_pairdiff.mean(0), 2), 
    f"[{np.round(np.quantile(cutpoint1_pairdiff, 0.025), 2)}, {np.round(np.quantile(cutpoint1_pairdiff, 0.975), 2)}]"
)

In [ ]:
_ = arviz.plot_posterior(fit_pairdiff["fit"], var_names=["fixed_coefs"])

In [ ]:
_ = arviz.plot_posterior(fit_pairdiff["fit"], var_names=["subj_scale", "item_scale", "verb_scale"])

In [ ]:

_ = arviz.plot_posterior(fit_pairdiff["fit"], var_names=["sample_size", "interval_size"])

In [ ]:
fit_pairdiff_surprisal = fit_hmc(
    data_diff,
    fixed_formula=("~ 1 + C(PairType, levels=comparison_levels) * surprisal1_z * surprisal2_z"
                      " + C(PairType, levels=comparison_levels) * typicality1 * typicality2" 
                      " + C(PairType, levels=comparison_levels) * freq"),
    by_subj_formula="~ 1 + C(PairType, levels=comparison_levels)",
    by_verb_formula="~ 1 + C(PairType, levels=comparison_levels)",
    by_sense_formula="~ 1", 
    by_item_formula="~ 1", 
    max_init_iter=0,
    item_cols=["sentence1", "sentence2"],
    sense_cols = ["verb", "sense1", "sense2"],
    subj_cols=["rater_id"],
    verb_cols = ["verb"],
    output_dir="fits/pairdiff/",
    **sampler_params
)

In [ ]:
fit_pairdiff_surprisal["fixed_coefs"]

In [ ]:
print_fixed_coef_tabular(fit_pairdiff_surprisal, process_coef_name_pairdiff_surprisal)

In [ ]:
_ = arviz.plot_posterior(fit_pairdiff_surprisal["fit"], var_names=["fixed_coefs"])

In [ ]:
_ = arviz.plot_posterior(fit_pairdiff_surprisal["fit"], var_names=["subj_scale", "item_scale", "verb_scale"])

In [ ]:

_ = arviz.plot_posterior(fit_pairdiff_surprisal["fit"], var_names=["sample_size", "interval_size"])